<a href="https://colab.research.google.com/github/RodrigoGuedesDP/Computer_Vision/blob/main/Sesion07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales Convolucionales

## Max Pooling (Submuestreo máximo)

El **max pooling** es una operación de submuestreo utilizada habitualmente en redes neuronales convolucionales. Consiste en recorrer la imagen con una ventana de un tamaño determinado (p. ej. 2×2) y, para cada región, mantener únicamente el valor máximo. Esto reduce la resolución espacial de las características, hace la representación más compacta y proporciona invariancia a pequeñas translaciones.

En el siguiente ejemplo trabajaremos con una única imagen de un canal (es decir, una matriz 4×4) y aplicaremos una capa de max pooling con ventana 2×2 y paso (`stride`) 2. La salida tendrá la mitad de la altura y la mitad del ancho de la entrada.


In [ ]:
import torch
import torch.nn as nn

# Definimos una entrada de ejemplo: 1 imagen (batch), 1 canal, tamaño 4×4.
# Cada número se incrementa para poder ver claramente el efecto del pooling.
x = torch.tensor([[[[1.0, 2.0, 3.0, 4.0],
                    [5.0, 6.0, 7.0, 8.0],
                    [9.0,10.0,11.0,12.0],
                    [13.0,14.0,15.0,16.0]]]])

print(f"Forma de la entrada: {x.shape}")


Forma de la entrada: torch.Size([1, 1, 4, 4])


In [ ]:
# Creamos la capa de max pooling con ventana 2×2 y stride 2.
pool = nn.MaxPool2d(kernel_size=2, stride=2)


In [ ]:
# Aplicamos el max pooling a la imagen de entrada
y = pool(x)
print("Resultado del max pooling:")
print(y)
print(f"Forma de la salida: {y.shape}")


Resultado del max pooling:
tensor([[[[ 6.,  8.],
          [14., 16.]]]])
Forma de la salida: torch.Size([1, 1, 2, 2])


## Convolución 2D

Las **convoluciones** son el núcleo de las redes convolucionales. Se aplican filtros (o kernels) que se deslizan sobre la imagen para extraer patrones locales como bordes, texturas y formas. En PyTorch, una capa `nn.Conv2d` recibe un tensor de entrada de forma `(batch_size, canales, altura, ancho)` y aplica `out_channels` filtros de tamaño `kernel_size`.

A continuación definiremos un pequeño ejemplo con una entrada 4×4 de un solo canal y un filtro 3×3.

In [ ]:
import torch
import torch.nn as nn

# Entrada: 1 imagen, 1 canal, tamaño 4×4
x = torch.tensor([[[[1.0, 2.0, 3.0, 4.0],
                    [5.0, 6.0, 7.0, 8.0],
                    [9.0,10.0,11.0,12.0],
                    [13.0,14.0,15.0,16.0]]]])
print(f"Forma de la entrada: {x.shape}")


Forma de la entrada: torch.Size([1, 1, 4, 4])


In [ ]:
# Creamos una capa convolucional con un filtro 3×3
# in_channels=1 porque la entrada tiene 1 canal y out_channels=1 porque queremos un único mapa de salida.
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)

In [ ]:
# Aplicamos la convolución
y = conv(x)
print("Mapa de activaciones resultante:")
print(y)
print(f"Forma de la salida: {y.shape}")


Mapa de activaciones resultante:
tensor([[[[4.8553, 4.9304],
          [5.1555, 5.2306]]]], grad_fn=<ConvolutionBackward0>)
Forma de la salida: torch.Size([1, 1, 2, 2])


## Capas convolucionales y mapas de activación

Cuando encadenamos varias capas convolucionales, cada una produce un **mapa de activaciones** con una cierta altura y anchura.
El tamaño de salida `(H_out, W_out)` de una convolución sin padding se calcula mediante:

$$
H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} - \text{kernel_size}}{\text{stride}} \right\rfloor + 1
$$

$$
W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} - \text{kernel_size}}{\text{stride}} \right\rfloor + 1
$$


Esta fórmula te permite prever cómo disminuyen las dimensiones espaciales a medida que aplicas filtros. En el siguiente ejemplo crearemos un tensor de entrada aleatorio y veremos las dimensiones de salida tras aplicar dos capas convolucionales consecutivas.


In [ ]:
import torch
import torch.nn as nn

# Creamos una entrada aleatoria: lote de tamaño 1, 3 canales, 19×19 píxeles
x = torch.randn(1, 3, 19, 19)

# Primera capa convolucional: 3 canales de entrada → 7 canales de salida, kernel 5×5
conv1 = nn.Conv2d(in_channels=3, out_channels=7, kernel_size=5, bias=False)
feature_map1 = conv1(x)

print(f"Forma de la entrada: {x.shape}")
print(f"Forma de los filtros de la primera capa: {conv1.weight.shape}")
print(f"Forma del mapa de activaciones tras conv1: {feature_map1.shape}")


Forma de la entrada: torch.Size([1, 3, 19, 19])
Forma de los filtros de la primera capa: torch.Size([7, 3, 5, 5])
Forma del mapa de activaciones tras conv1: torch.Size([1, 7, 15, 15])


En este caso, la entrada tenía tamaño 19×19 y un `kernel_size` de 5 con `stride=1`. Por tanto, cada dimensión disminuye a **19 − 5 + 1 = 15**, lo que concuerda con la forma de salida `(1, 7, 15, 15)`.

Si añadimos otra capa convolucional con `kernel_size=3` y `in_channels=7`, el tamaño volverá a disminuir en 2 píxeles por dimensión (15 − 3 + 1 = 13).


In [ ]:
# Segunda capa convolucional: toma los 7 canales de salida de conv1 y produce 2 canales de salida
conv2 = nn.Conv2d(in_channels=7, out_channels=2, kernel_size=3, bias=False)
feature_map2 = conv2(feature_map1)

print(f"Forma de los filtros de la segunda capa: {conv2.weight.shape}")
print(f"Forma del mapa de activaciones tras conv2: {feature_map2.shape}")


Forma de los filtros de la segunda capa: torch.Size([2, 7, 3, 3])
Forma del mapa de activaciones tras conv2: torch.Size([1, 2, 13, 13])


## Modelos preentrenados: AlexNet y VGG16

Las bibliotecas de visión por computadora modernas incluyen modelos previamente entrenados sobre grandes conjuntos de datos como **ImageNet**. Estos modelos han aprendido representaciones útiles que pueden reutilizarse para tareas nuevas (lo que se conoce como _transfer learning_).

Usaremos `torchvision.models` para cargar dos arquitecturas clásicas:

- **AlexNet**: propuesto en 2012, con cinco capas convolucionales y tres capas completamente conectadas.
- **VGG16**: introducido en 2014, con una arquitectura más profunda basada únicamente en filtros 3×3.

También utilizaremos la función `summary` de `torchinfo` para resumir las dimensiones de entrada y salida de cada capa.


In [ ]:
import torchvision.models as models

# Cargamos los modelos AlexNet y VGG16 preentrenados en ImageNet
alex = models.alexnet(weights='IMAGENET1K_V1')
vgg  = models.vgg16(weights='IMAGENET1K_V1')


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 156MB/s]


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:07<00:00, 71.0MB/s]


In [ ]:
# Instalamos e importamos torchinfo para visualizar un resumen de los modelos
!pip install -q torchinfo
from torchinfo import summary

# Mostramos el resumen de AlexNet para una entrada de una imagen (batch=1) de 3 canales y 224×224 píxeles
summary(alex, input_size=(1, 3, 224, 224))


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

In [ ]:
# Mostramos el resumen de VGG16 para la misma entrada
summary(vgg, input_size=(1, 3, 224, 224))


Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

## Implementación de AlexNet desde cero

Para comprender mejor el funcionamiento interno de AlexNet podemos implementar la red manualmente utilizando `torch.nn`. La versión original de 2012 constaba de:

1. **Cinco capas convolucionales** con funciones de activación ReLU y capas de max pooling intercaladas.
2. **Tres capas completamente conectadas**, con funciones de activación ReLU y dropout entre ellas.
3. Un clasificador final que produce probabilidades para 1000 clases de ImageNet.

A continuación se muestra una implementación fiel a la arquitectura original. Observa cómo se utiliza `nn.Sequential` para encadenar las capas y cómo se aplanan (`torch.flatten`) las salidas antes del clasificador.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    """
    Implementación de la arquitectura AlexNet de 2012.

    Args:
        num_classes (int): número de clases de salida. Por defecto 1000 (ImageNet).
        in_channels (int): número de canales de entrada de la imagen. Por defecto 3.
    """
    def __init__(self, num_classes: int = 1000, in_channels: int = 3):
        super().__init__()

        # Bloques convolucionales: cada bloque está formado por una convolución,
        # una activación ReLU y opcionalmente un max pooling.
        self.features = nn.Sequential(
            # Bloque 1: kernel 11×11, stride 4, padding 2
            nn.Conv2d(in_channels, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            # Bloque 2: kernel 5×5
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            # Bloque 3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # Bloque 4
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            # Bloque 5
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        # Clasificador: aplanamos las características y pasamos por capas densas con dropout
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        # Propagación hacia adelante: obtenemos las características y las aplanamos
        x = self.features(x)
        x = torch.flatten(x, 1)  # aplanar excepto la dimensión del batch
        x = self.classifier(x)
        return x

# Instanciamos el modelo para 1000 clases
model = AlexNet(num_classes=1000)

# Creamos una entrada dummy: lote de 1 imagen RGB de 224×224
input_tensor = torch.randn(1, 3, 224, 224)

# Calculamos la salida del modelo
y = model(input_tensor)
print(f"Forma de la salida del modelo: {y.shape}")  # Debería ser (1, 1000)

# Calculamos y mostramos el número total de parámetros (en millones)
params = sum(p.numel() for p in model.parameters()) / 1e6
print(f"Total de parámetros de AlexNet: {params:.1f} millones")


Forma de la salida del modelo: torch.Size([1, 1000])
Total de parámetros de AlexNet: 61.1 millones


En la implementación anterior observamos que la salida del modelo tiene forma `(1, 1000)`,
correspondiente al número de clases de ImageNet. Además, el modelo contiene alrededor de **61,1 M** de parámetros aprendibles.

Este recuento de parámetros se obtiene sumando todos los pesos y sesgos de las capas convolucionales y totalmente conectadas.


In [ ]:
summary(model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         